In [ ]:
# Run this cell to Download dataset
!gdown 18eEEtko63fth0HEIFdxEdbjR4DfXiPa7

Downloading...
From: https://drive.google.com/uc?id=18eEEtko63fth0HEIFdxEdbjR4DfXiPa7
To: /content/processed_purchase_propensity_data_v1.parquet
100% 58.0M/58.0M [00:01<00:00, 41.6MB/s]


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet("./processed_purchase_propensity_data_v1.parquet")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2933439 entries, 0 to 2933438
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   user_id               int64         
 1   product_id            int64         
 2   event_timestamp       datetime64[ns]
 3   created_timestamp     datetime64[us]
 4   category_code_level1  object        
 5   category_code_level2  object        
 6   brand                 object        
 7   event_weekday         int64         
 8   price                 float64       
 9   activity_count        int64         
 10  is_purchased          int64         
dtypes: datetime64[ns](1), datetime64[us](1), float64(1), int64(5), object(3)
memory usage: 246.2+ MB


In [ ]:
df

,user_id,product_id,event_timestamp,created_timestamp,category_code_level1,category_code_level2,brand,event_weekday,price,activity_count,is_purchased
0,515903856,2601552,2019-11-17 00:11:39,2026-01-18 22:17:22.150556,unknown,unknown,gorenje,6,486.24,6,0
1,516301799,12702930,2019-11-12 15:40:15,2026-01-18 22:17:22.150556,unknown,unknown,cordiant,1,35.78,2,0
2,516301799,12702930,2019-11-12 15:41:46,2026-01-18 22:17:22.150556,unknown,unknown,cordiant,1,35.78,6,0
3,516301799,12702930,2019-11-12 15:42:05,2026-01-18 22:17:22.150556,unknown,unknown,cordiant,1,35.78,8,0
4,561066382,3800966,2019-11-15 23:36:25,2026-01-18 22:17:22.150556,appliances,iron,elenberg,4,20.57,2,0
...,...,...,...,...,...,...,...,...,...,...,...
2933434,569255486,3300427,2019-11-21 14:21:54,2026-01-18 22:17:22.150556,unknown,unknown,redmond,3,142.84,27,0
2933435,569255486,3300427,2019-11-21 14:30:14,2026-01-18 22:17:22.150556,unknown,unknown,redmond,3,142.84,29,0
2933436,569255486,3300427,2019-11-21 14:30:53,2026-01-18 22:17:22.150556,unknown,unknown,redmond,3,142.84,31,0
2933437,569255486,3300427,2019-11-21 14:32:14,2026-01-18 22:17:22.150556,unknown,unknown,redmond,3,142.84,35,1


In [ ]:
import numpy as np
import json
import xgboost as xgb
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from typing import Tuple, Dict, Any

### GLOBAL CONFIGURATION


In [ ]:
CONFIG = {
    "file_path": "/content/processed_purchase_propensity_data_v1.parquet",
    "target_column": "is_purchased",
    "output_file": "xgboost_final_result.json",
    "model_path": "xgboost_model_final.pkl",
    "test_size": 0.2,
    "random_state": 89,
    "model_params": {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'n_estimators': 800,
        'max_depth': 4,
        'learning_rate': 0.03,
        'subsample': 0.8,
        'n_jobs': -1,
        'tree_method': 'hist', # Enable GPU training
    }
}

### DATA LOADING

In [ ]:
def load_data(file_path: str) -> pd.DataFrame:
  print(f"[INFO] Loading data from: {file_path}")
  try:
    df = pd.read_parquet(file_path)
    return df
  except FileNotFoundError:
    raise FileNotFoundError(f"[ERROR] File not found at {file_path}")

### FEATURE ENGINEERING

In [ ]:
def final_FE(df: pd.DataFrame) -> pd.DataFrame:
  df_fe = df.copy()

  # log-transform
  df_fe['price_log'] = np.log1p(df_fe['price'])
  df_fe['activity_log'] = np.log1p(df_fe['activity_count'])

  # datetime features
  df_fe['event_ts'] = pd.to_datetime(df_fe['event_timestamp'])
  df_fe['event_hour'] = df_fe['event_ts'].dt.hour
  df_fe['is_weekend'] = (df_fe['event_ts'].dt.weekday >= 5).astype(int)

  # user-level stats
  user_stats = df_fe.groupby('user_id').agg(
      user_total_events=('is_purchased', 'count'),
      user_purchase_rate=('is_purchased', 'mean')
  ).reset_index()

  df_fe = df_fe.merge(user_stats, on='user_id', how='left')

  # product-level stats
  product_stats = df_fe.groupby('product_id').agg(
      product_total_events=('is_purchased', 'count'),
      product_purchase_rate=('is_purchased', 'mean')
  ).reset_index()

  df_fe = df_fe.merge(product_stats, on='product_id', how='left')

  # log-transform counts
  df_fe['user_total_log'] = np.log1p(df_fe['user_total_events'])
  df_fe['product_total_log'] = np.log1p(df_fe['product_total_events'])

  # recency features
  df_fe['hours_since_event'] = (
      df_fe['event_ts'].max() - df_fe['event_ts']
  ).dt.total_seconds() / 3600

  df_fe['hours_log'] = np.log1p(df_fe['hours_since_event'])

  return df_fe


### filtering numeric columns

In [ ]:
def drop_non_numeric_columns(df: pd.DataFrame) -> pd.DataFrame:
  return df.select_dtypes(include=['int', 'float', 'bool'])


### PIPELINE CONSTRUCTION

In [ ]:
def build_pipeline(numeric_cols: list, categorical_cols: list, model_params: Dict) -> ImbPipeline:
  # Define Full Pipeline
  pipeline = ImbPipeline(steps=[
      ('classifier', xgb.XGBClassifier(**model_params, random_state=CONFIG["random_state"]))
  ])

  return pipeline


### TRAINING EXECUTION

In [ ]:
def train_model(df: pd.DataFrame, target_col: str) -> Tuple[Any, Any, Any]:
  """
  Splits data, builds pipeline, and fits the model.
  """
  print("[PROCESS] Splitting data and training model...")

  X = df.drop(columns=[target_col])
  y = df[target_col]

  # Split Data
  X_train, X_test, y_train, y_test = train_test_split(
      X, y,
      test_size=CONFIG["test_size"],
      random_state=CONFIG["random_state"],
      stratify=y
  )

  # Identify Column Types
  num_cols = X.select_dtypes(include=['number']).columns.tolist()
  cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

  print(f"[INFO] Numeric Features: {len(num_cols)}")
  print(f"[INFO] Categorical Features: {len(cat_cols)}")

  # Build and Fit Pipeline
  pipeline = build_pipeline(num_cols, cat_cols, CONFIG["model_params"])
  pipeline.fit(X_train, y_train)

  return pipeline, X_test, y_test

### OPTIMIZATION & EVALUATION

In [ ]:
def find_optimal_threshold(pipeline, X_test, y_test) -> float:
  """
  Finds the threshold that maximizes F1-Score.
  """
  print("[PROCESS] Tuning decision threshold...")
  probs = pipeline.predict_proba(X_test)[:, 1]
  precisions, recalls, thresholds = precision_recall_curve(y_test, probs)

  f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
  f1_scores = np.nan_to_num(f1_scores)

  best_idx = np.argmax(f1_scores)
  best_thresh = thresholds[best_idx]

  print(f"[INFO] Optimal Threshold found: {best_thresh:.4f} (Max F1: {f1_scores[best_idx]:.4f})")
  return best_thresh

### CALCULATE METRICS

In [ ]:
def calculate_metrics(pipeline, X_test, y_test, threshold: float) -> Dict[str, float]:
  """
  Calculates final metrics based on the selected threshold.
  """
  probs = pipeline.predict_proba(X_test)[:, 1]
  preds = (probs >= threshold).astype(int)

  return {
      "accuracy": accuracy_score(y_test, preds),
      "f1": f1_score(y_test, preds),
      "recall": recall_score(y_test, preds),
      "precision": precision_score(y_test, preds),
      "roc_auc": roc_auc_score(y_test, probs)
  }


### SAVE FILE

In [ ]:

def save_artifacts(pipeline, metrics: Dict, params: Dict):
  """
  Saves the model and results to disk.
  """
  # Save Model
  joblib.dump(pipeline, CONFIG["model_path"])
  print(f"[SAVE] Model saved to: {CONFIG['model_path']}")

  # Save Metrics
  output_data = {
      "metrics": metrics,
      "params": params,
      "config": {k:v for k,v in CONFIG.items() if k != "model_params"}
  }
  with open(CONFIG["output_file"], "w", encoding="utf-8") as f:
      json.dump(output_data, f, indent=4, ensure_ascii=False)
  print(f"[SAVE] Metrics saved to: {CONFIG['output_file']}")

### MAIN CONTROLLER

In [ ]:
def main():
  # Load Data
  df = load_data(CONFIG["file_path"])

  # Process General Features (Time, Price, IDs)
  #df = process_datetime_and_price(df)

  # Add User History Features (New & Critical Step)
  #df = add_user_history_features(df)
  df = final_FE(df)
  df = drop_non_numeric_columns(df)

  # Train Model
  pipeline, X_test, y_test = train_model(df, CONFIG["target_column"])

  # Optimize Threshold
  best_threshold = find_optimal_threshold(pipeline, X_test, y_test)

  # Evaluate
  metrics = calculate_metrics(pipeline, X_test, y_test, best_threshold)

  # Report & Save
  print("\n" + "="*40)
  print("FINAL RESULTS (Optimized)")
  print("-" * 40)
  for k, v in metrics.items():
    print(f"{k.ljust(15)}: {v:.4f}")
  print("="*40)

  final_params = CONFIG["model_params"].copy()
  final_params['optimal_threshold'] = float(best_threshold)
  save_artifacts(pipeline, metrics, final_params)

if __name__ == "__main__":
  main()

[INFO] Loading data from: /content/processed_purchase_propensity_data_v1.parquet
[PROCESS] Splitting data and training model...
[INFO] Numeric Features: 17
[INFO] Categorical Features: 0
[PROCESS] Tuning decision threshold...
[INFO] Optimal Threshold found: 0.3705 (Max F1: 0.7434)

FINAL RESULTS (User History + SMOTE + Optimized)
----------------------------------------
accuracy       : 0.8491
f1             : 0.7434
recall         : 0.8400
precision      : 0.6668
roc_auc        : 0.9348
[SAVE] Model saved to: xgboost_model_final.pkl
[SAVE] Metrics saved to: xgboost_final_result.json


### READ FILE Json AND pkl

In [ ]:
import json
import joblib

# Read JSON file
json_file_path = "xgboost_final_result.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)
print(f"Content of {json_file_path}:\n{json.dumps(json_data, indent=4, ensure_ascii=False)}")

print("\n" + "="*50 + "\n")

# Read PKL file (model)
pkl_file_path = "xgboost_model_final.pkl"
loaded_model = joblib.load(pkl_file_path)
print(f"Successfully loaded model from {pkl_file_path}:\n{loaded_model}")

Content of xgboost_final_result.json:
{
    "metrics": {
        "accuracy": 0.8491259408748772,
        "f1": 0.743431884057971,
        "recall": 0.8400112663509468,
        "precision": 0.6667706529820676,
        "roc_auc": 0.9348168372960998
    },
    "params": {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "n_estimators": 800,
        "max_depth": 4,
        "learning_rate": 0.03,
        "subsample": 0.8,
        "n_jobs": -1,
        "tree_method": "hist",
        "optimal_threshold": 0.3705163300037384
    },
    "config": {
        "file_path": "/content/processed_purchase_propensity_data_v1.parquet",
        "target_column": "is_purchased",
        "output_file": "xgboost_final_result.json",
        "model_path": "xgboost_model_final.pkl",
        "test_size": 0.2,
        "random_state": 89
    }
}


Successfully loaded model from xgboost_model_final.pkl:
Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=None, booster

In [ ]:
df = load_data(CONFIG["file_path"])

df = final_FE(df)
df = drop_non_numeric_columns(df)


df.info()

[INFO] Loading data from: /content/processed_purchase_propensity_data_v1.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2933439 entries, 0 to 2933438
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   user_id                int64  
 1   product_id             int64  
 2   event_weekday          int64  
 3   price                  float64
 4   activity_count         int64  
 5   is_purchased           int64  
 6   price_log              float64
 7   activity_log           float64
 8   event_hour             int32  
 9   is_weekend             int64  
 10  user_total_events      int64  
 11  user_purchase_rate     float64
 12  product_total_events   int64  
 13  product_purchase_rate  float64
 14  user_total_log         float64
 15  product_total_log      float64
 16  hours_since_event      float64
 17  hours_log              float64
dtypes: float64(9), int32(1), int64(8)
memory usage: 391.7 MB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,2933439.0,5.408460e+08,2.272728e+07,7.438304e+07,5.176743e+08,5.399510e+08,5.636370e+08,5.799677e+08
product_id,2933439.0,8.455282e+06,1.377809e+07,1.000978e+06,1.005100e+06,3.600163e+06,1.270302e+07,1.000282e+08
event_weekday,2933439.0,3.777605e+00,1.890691e+00,0.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00,6.000000e+00
price,2933439.0,2.905062e+02,3.357733e+02,0.000000e+00,7.439000e+01,1.687000e+02,3.497900e+02,2.574070e+03
activity_count,2933439.0,8.280268e+00,1.148055e+01,1.000000e+00,2.000000e+00,5.000000e+00,9.000000e+00,4.180000e+02
is_purchased,2933439.0,2.602181e-01,4.387536e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
price_log,2933439.0,5.076557e+00,1.176949e+00,0.000000e+00,4.322675e+00,5.134032e+00,5.860188e+00,7.853632e+00
activity_log,2933439.0,1.876440e+00,7.512437e-01,6.931472e-01,1.098612e+00,1.791759e+00,2.302585e+00,6.037871e+00
event_hour,2933439.0,1.383522e+01,6.631623e+00,0.000000e+00,1.100000e+01,1.500000e+01,1.900000e+01,2.300000e+01
is_weekend,2933439.0,4.201628e-01,4.935850e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
df

,user_id,product_id,event_weekday,price,activity_count,is_purchased,price_log,activity_log,event_hour,is_weekend,user_total_events,user_purchase_rate,product_total_events,product_purchase_rate,user_total_log,product_total_log,hours_since_event,hours_log
0,515903856,2601552,6,486.24,6,0,6.188757,1.945910,0,1,19,0.052632,27,0.185185,2.995732,3.332205,342.803611,5.840071
1,516301799,12702930,1,35.78,2,0,3.604954,1.098612,15,0,5,0.200000,3702,0.225284,1.791759,8.216899,447.326944,6.105523
2,516301799,12702930,1,35.78,6,0,3.604954,1.945910,15,0,5,0.200000,3702,0.225284,1.791759,8.216899,447.301667,6.105466
3,516301799,12702930,1,35.78,8,0,3.604954,2.197225,15,0,5,0.200000,3702,0.225284,1.791759,8.216899,447.296389,6.105455
4,561066382,3800966,4,20.57,2,0,3.071303,1.098612,23,0,6,0.666667,315,0.253968,1.945910,5.755742,367.390833,5.909144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933434,569255486,3300427,3,142.84,27,0,4.968702,3.332205,14,0,6,0.166667,56,0.196429,1.945910,4.043051,232.632778,5.453751
2933435,569255486,3300427,3,142.84,29,0,4.968702,3.401197,14,0,6,0.166667,56,0.196429,1.945910,4.043051,232.493889,5.453156
2933436,569255486,3300427,3,142.84,31,0,4.968702,3.465736,14,0,6,0.166667,56,0.196429,1.945910,4.043051,232.483056,5.453110
2933437,569255486,3300427,3,142.84,35,1,4.968702,3.583519,14,0,6,0.166667,56,0.196429,1.945910,4.043051,232.460556,5.453013
